In [306]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

When features has some sort of collinearity, typical linear regression ca be fixed by applying ridge penalty.

$B = (X^{T}X + λI)^{-1}y$
$λ - ridge penalty

Data need to be standardizeds to make the ridge penalty the and features on the same scale.

In [307]:
teams = pd.read_csv("teams.csv")

In [308]:
teams

,team,year,athletes,events,age,height,weight,prev_medals,medals
0,AFG,1964,8,8,22.0,161.0,64.2,0.0,0
1,AFG,1968,5,5,23.2,170.2,70.0,0.0,0
2,AFG,1972,8,8,29.0,168.3,63.8,0.0,0
3,AFG,1980,11,11,23.6,168.4,63.2,0.0,0
4,AFG,2004,5,5,18.6,170.8,64.8,0.0,0
...,...,...,...,...,...,...,...,...,...
2009,ZIM,2000,26,19,25.0,179.0,71.1,0.0,0
2010,ZIM,2004,14,11,25.1,177.8,70.5,0.0,3
2011,ZIM,2008,16,15,26.1,171.9,63.7,3.0,4
2012,ZIM,2012,9,8,27.3,174.4,65.2,4.0,0


In [309]:
#set random_state=1 to persist the same instance every time train_test_split is run
train, test = train_test_split(teams, test_size=0.2, random_state=1)

In [310]:
predictors = ["athletes", "events"]
target = ["medals"]

In [311]:
X = train[predictors].copy()
X

,athletes,events
1322,6,6
1872,119,80
953,4,4
1117,2,2
1993,43,25
...,...,...
1791,40,25
1096,36,23
1932,719,245
235,13,11


In [312]:
Y = train[target].copy()
Y

,medals
1322,0
1872,5
953,0
1117,0
1993,0
...,...
1791,1
1096,1
1932,264
235,0


In [313]:
x_mean = X.mean()
x_std = X.std()

In [314]:
x_mean

athletes    74.409063
events      35.990068
dtype: float64

In [315]:
x_std

athletes    127.250043
events       48.978737
dtype: float64

In [316]:
#Scale the data to make standardard deviation equal to 1 and mean equal to 0
X = (X - x_mean) / x_std

In [317]:
X.describe()

,athletes,events
count,1.611000e+03,1.611000e+03
mean,-2.370681e-17,-9.923781e-18
std,1.000000e+00,1.000000e+00
min,-5.768883e-01,-7.143930e-01
25%,-5.297371e-01,-6.123079e-01
50%,-4.197174e-01,-4.489717e-01
75%,-2.679027e-02,1.839560e-01
max,6.008571e+00,4.634867e+00


In [318]:
X["intercept"] = 1
X = X[["intercept"] + predictors]

In [319]:
X

,intercept,athletes,events
1322,1,-0.537596,-0.612308
1872,1,0.350420,0.898552
953,1,-0.553313,-0.653142
1117,1,-0.569030,-0.693976
1993,1,-0.246829,-0.224384
...,...,...,...
1791,1,-0.270405,-0.224384
1096,1,-0.301839,-0.265219
1932,1,5.065546,4.267361
235,1,-0.482586,-0.510223


In [320]:
alpha = 2
I = np.identity(X.shape[1])

In [321]:
I[0][0] = 0
I

array([[0., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [322]:
penalty = alpha * I
penalty

array([[0., 0., 0.],
       [0., 2., 0.],
       [0., 0., 2.]])

In [323]:
#Calculate for ridge regression coefficients using the TRAIN sets
B = np.linalg.inv(X.T @ X + penalty) @ X.T @ Y

In [324]:
#COEFFICIENTS
B.index = ["intercept", "athletes", "events"]
B

,medals
intercept,10.691496
athletes,61.857734
events,-34.632920


In [325]:
#Create the TEST sets
test_X = test[predictors]
test_X

,athletes,events
309,4,4
285,150,85
919,56,41
120,93,62
585,36,30
...,...,...
541,26,16
1863,50,43
622,67,55
1070,2,2


In [326]:
#Scale the test data using the Mean and Standard Deviation of the Train set
test_X = (test_X - x_mean) / x_std

In [327]:
test_X["intercept"] = 1
test_X = test_X[["intercept"] + predictors]

In [328]:
test_X

,intercept,athletes,events
309,1,-0.553313,-0.653142
285,1,0.594035,1.000637
919,1,-0.144668,0.102288
120,1,0.146098,0.531045
585,1,-0.301839,-0.122299
...,...,...,...
541,1,-0.380425,-0.408138
1863,1,-0.191820,0.143122
622,1,-0.058224,0.388126
1070,1,-0.569030,-0.693976


In [329]:
#CALCULATE PREDICTIONS by Mulitplying the test_X features by the calculated B from the train set
predictions = test_X @ B

In [330]:
predictions

,medals
309,-0.914959
285,12.782156
919,-1.799893
120,1.337116
585,-3.744014
...,...
541,1.294285
1863,-6.130765
622,-6.352080
1070,-0.472980


Verify manual ridge regression implementation with Scikit-learn results

In [331]:
from sklearn.linear_model import Ridge

In [332]:
#Initialize the ridge regression algorith with the the parameter alpha
ridge = Ridge(alpha=alpha)

In [333]:
#Fit the data with the model
ridge.fit(X[predictors], Y)

Ridge(alpha=2)

In [334]:
ridge.coef_

array([[ 61.85773366, -34.63292036]])

In [335]:
ridge.intercept_

array([10.69149597])

The manual implementation results for coefficient are matching with the sklearn results

In [336]:
sklearn_predictions = ridge.predict(test_X[predictors])

In [337]:
#Verify if manual predictions are the same with sklearn predictions
predictions - sklearn_predictions

,medals
309,8.348877e-14
285,-3.534950e-13
919,-2.051692e-13
120,-3.286260e-13
585,-1.483258e-13
...,...
541,2.309264e-14
1863,-2.797762e-13
622,-3.730349e-13
1070,1.048051e-13


FINAL IMPLEMENTATION

In [338]:
def ridge_fit(train, predictors, target, alpha):
    X = train[predictors].copy()
    Y = train[target].copy()

    x_mean = X.mean()
    x_std = X.std()

    X = (X - x_mean) / x_std
    X["intercept"] = 1
    X = X[["intercept"] + predictors]

    penalty = alpha * np.identity(X.shape[1])
    penalty[0][0] = 0

    B = np.linalg.inv(X.T @ X + penalty) @ X.T @ Y
    B.index = ["intercept", "athletes", "events"]
    return B, x_mean, x_std

In [339]:
def ridge_predict(test, predictors, x_mean, x_std, B):
    test_X = test[predictors]
    test_X = (test_X - x_mean) / x_std
    test_X["intercept"] = 1
    test_X = test_X[["intercept"] + predictors]

    predictions = test_X @ B
    return predictions

In [340]:
#Identify the average difference between the actual and predicted Y values (Mean Absolute Error)

from sklearn.metrics import mean_absolute_error

errors = []
alphas= [10 ** i for i in range(-2, 4)]

Mean Absolute Error

$\frac{1}n \sum |y - \hat{y}|$

In [341]:
alphas

[0.01, 0.1, 1, 10, 100, 1000]

In [342]:
for alpha in alphas:
    B, x_mean, x_std = ridge_fit(train, predictors, target, alpha)
    predictions = ridge_predict(test, predictors, x_mean, x_std, B)
    errors.append(mean_absolute_error(test[target], predictions))

In [343]:
#The corresponding value to the lowest mean absolute error is the optimal ridge penalty.
# In this case, the lowest MAE is 6.11 and the corresponding penalty is 10.
errors

[6.309640830161113,
 6.306044331952916,
 6.272283376431602,
 6.114051204717718,
 7.156811236590466,
 6.9780545895757315]

FINAL VALUES

In [344]:
print(B)
print(f"Ridge: {10}")

              medals
intercept  10.691496
athletes   12.286109
events      8.439492
Ridge: 10
